# Core Imports

In [24]:
# Custom Imports
from polysaccharide import general
from polysaccharide.general import optional_in_place
from polysaccharide.extratypes import ResidueSmarts

from polysaccharide.molutils import reactions
from polysaccharide.molutils.rdmol.rdtypes import *
from polysaccharide.molutils.rdmol import rdcompare, rdconvert, rdkdraw, rdcompare, rdprops, rdbond, rdlabels

from polysaccharide.polymer import monomer as monoutils
from polysaccharide.polymer.monomer import MonomerInfo

from polysaccharide.polymer import building
import mbuild as mb

# Generic Imports
import re
from functools import partial, cached_property
from collections import defaultdict
from itertools import combinations, chain
from ast import literal_eval

# Numeric imports
import pandas as pd
import numpy as np

# File I/O
from pathlib import Path
import csv, json, openpyxl

# Typing and Subclassing
from typing import Any, Callable, ClassVar, Generator, Iterable, Optional, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod, abstractproperty
from openmm.unit import Unit, Quantity

# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdChemReactions

# Static Paths
RAW_DATA_PATH  = Path('raw_monomer_data')
PROC_DATA_PATH = Path('processed_monomer_data')
RXN_FILES_PATH = Path('rxn_smarts')
MONO_INFO_DIR  = Path('monomer_files')

# Building PDB files from monomer fragments

In [25]:
# defining reacting functional groups
reaction_pairs = {
    'NIPU' : ('cyclocarbonate', 'amine'),
    'urethane' : ('isocyanate', 'hydroxyl')
}

In [26]:
from polysaccharide.polymer.exceptions import SubstructMatchFailedError

pdb_path = Path('pdb_files')
pdb_path.mkdir(exist_ok=True)

DOP = 10
chemistries = ('urethane',)

failed = defaultdict(defaultdict)
match_failed = {}
too_long = {}
for chemistry in chemistries:
    mono_src_dir = MONO_INFO_DIR / chemistry
    pdb_out_dir = pdb_path / chemistry
    pdb_out_dir.mkdir(exist_ok=True)

    for mono_path in mono_src_dir.iterdir():
        mono_info = MonomerInfo.from_file(mono_path)
        mono_subset = {
            resname : SMARTS
                for resname, SMARTS in mono_info.SMARTS.items()
                    if 'DIMER' not in resname # withdraw dimers
        }

        try:
            polymer = building.build_linear_polymer(mono_subset, DOP=DOP, sequence='AB')  
            savepath = pdb_out_dir / f'{mono_path.stem}.pdb'
            polymer.save(general.asstrpath(savepath))
        except SubstructMatchFailedError:
            failed[chemistry]['match_failed'][mono_path.stem] = mono_info
        except OSError:
            failed[chemistry]['too_long'][mono_path.stem] = mono_info

/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/mbuild/conversion.py:1736: UserWarning: The bond orders will be guessed using pybelOBMol.PerceviedBondOrders()
  warn(


In [ ]:
def show(mono_info : MonomerInfo) -> None:
    for resname, smarts in mono_info.SMARTS.items():
        print(resname)
        display(Chem.MolFromSmarts(smarts))

mono_info = match_failed['urethane_1']
show(mono_info)

# Generating managed polymer collections 

In [34]:
from polysaccharide.polymer.management import PolymerManager

coll_path = Path('Collections')
coll_path.mkdir(exist_ok=True)

for chemistry in chemistries:
    chem_path = coll_path / chemistry
    chem_path.mkdir(exist_ok=True)

    mgr = PolymerManager(chem_path)
    mgr.populate_collection(pdb_path / chemistry, MONO_INFO_DIR / chemistry)